In [12]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [13]:
import os, re
import pandas as pd
import psycopg2
import pandas as pd

from sql_queries import *


# CURRENT DIRECTORY (PLACE SCRIPT IN /home/mydirectory)

path_name2 = os.path.dirname('/home/workspace/data/song_data/') 
cd = os.path.abspath(path_name2)

i = 0
columns = range(1,100)
dfList = []

for root, dirs, files in os.walk(cd):
    for fname in files:
        if re.match("^.*.json$", fname):
            frame = pd.read_json(os.path.join(root, fname), lines=True )
            frame['key'] = "file{}".format(i)
            dfList.append(frame)    
            i += 1

df = pd.concat(dfList ,sort=True)

In [14]:
df.head()

artist_id  artist_latitude              artist_location  \
0  ARD7TVE1187B99BFB1              NaN              California - LA   
0  ARNTLGG11E2835DDB9              NaN                                
0  AR8ZCNI1187B9A069B              NaN                                
0  AR10USD1187B99F3F1              NaN  Burlington, Ontario, Canada   
0  ARMJAGH1187FB546F3         35.14968                  Memphis, TN   

   artist_longitude            artist_name   duration    key  num_songs  \
0               NaN                 Casual  218.93179  file0          1   
0               NaN                    Clp  266.39628  file1          1   
0               NaN       Planet P Project  269.81832  file2          1   
0               NaN  Tweeterfriendly Music  189.57016  file3          1   
0         -90.04892           The Box Tops  148.03546  file4          1   

              song_id                              title  year  
0  SOMZWCG12A8C13C480                   I Didn't Mean To     0  
0  SOUDSGM12AC9618304  Insatiable (Instrumental Version)     0  
0  SOIAZJW12AB01853F1                         Pink World  1984  
0  SOHKNRJ12A6701D1F8                       Drop of Rain     0  
0  SOCIWDW12A8C13D406                          Soul Deep  1969

In [15]:
artist_data = df[["artist_id", "artist_name", "artist_location" ,"artist_latitude" , "artist_longitude"]]


artist_data = artist_data.rename({'artist_name': 'name', 'artist_location': 'location','artist_latitude': 'latitude', 'artist_longitude': 'longitude'}, axis='columns')

# sorting by artist_id
artist_data.sort_values("artist_id", inplace = True) 
  
# dropping ALL duplicte values 
artist_data.drop_duplicates(subset ="artist_id", 
                     keep = False, inplace = True) 

In [16]:
artist_data.count()

artist_id    66
name         66
location     66
latitude     30
longitude    30
dtype: int64

In [17]:
artist_data.to_csv('/home/workspace/data/song_data/artist_data.csv', sep='\t', header=True, index=False)


In [18]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
copy_query = "COPY artists FROM '/home/workspace/data/song_data/artist_data.csv' DELIMITER '\t' CSV HEADER ; "  # Replace your table name in place of mem_info
cur.execute('truncate table artists')
cur.execute(copy_query)
conn.commit()

In [19]:
song_data = df[["song_id", "title", "artist_id" ,"year" , "duration"]]

song_data = song_data.rename({'song_id': 'song_id','artist_id' :'artist_id' , 'title': 'title','year': 'year', 'duration': 'duration'}, axis='columns')


song_data.drop_duplicates(subset ="song_id", keep = False, inplace = True)

song_data.head()

song_id                                        title  \
0  SOUDSGM12AC9618304            Insatiable (Instrumental Version)   
0  SOIAZJW12AB01853F1                                   Pink World   
0  SOHKNRJ12A6701D1F8                                 Drop of Rain   
0  SOYMRWW12A6D4FAB14  The Moon And I (Ordinary Day Album Version)   
0  SOMJBYD12A6D4F8557                        Keepin It Real (Skit)   

            artist_id  year   duration  
0  ARNTLGG11E2835DDB9     0  266.39628  
0  AR8ZCNI1187B9A069B  1984  269.81832  
0  AR10USD1187B99F3F1     0  189.57016  
0  ARKFYS91187B98E58F     0  267.70240  
0  ARD0S291187B9B7BF5     0  114.78159

In [20]:
song_data.to_csv('/home/workspace/data/song_data/song_data.csv', sep='\t', header=True, index=False)

In [21]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
copy_query1 = "COPY songs FROM '/home/workspace/data/song_data/song_data.csv' DELIMITER '\t' CSV HEADER ; "
cur.execute('truncate table songs' )
cur.execute(copy_query1)
cur.execute('commit')